In [1]:
from typing import *
import pandas as pd
import numpy as np

# 1. Laden des Wortdatensatzes
Um mit den Daten von SUBTLEX arbeiten zu können sind einige Schritte der Aufbereitung notwendig.
Zu Beginn wird die (Excel-)Datei eingelesen und in einen Pandas Dataframe konvertiert

In [2]:
df = pd.read_excel('SUBTLEX-us.xlsx')

# 2. Entfernen von Wörtern mit unbrauchbaren Wortklassen
Einige der Wörter wurden Wortklassen zugeordnert welche für unsere Aufgaben uninteressant sind oder nicht sinnvoll in die Aufgabenstellung integriert werden können.
So können z.B. einzelne Buchstaben (Letter) genau so wenig für die Aufgabenerstellung verwendet werden wie Wörter denen keine Klasse zugeordnert werden konnte (Unclssified und NaN).

In [3]:
df = df[(~df['Dom_PoS_SUBTLEX'].isin(['Letter', 'Unclassified', 'To', 'Not', 'Ex', np.nan]))]

# 3. Entfernen von Wörtern mit nicht eindeutiger Wortklasse
Da Wörter mehreren Wortklassen zugehörig sein können (je nach konkreter Verwendung) wird mit dem Wert in Spalte "Percentage_dom_PoS" angeben wie häufig das Wort die in der Spalte "Dom_PoS_SUBTLEX" angegebene klasse tatsächlich zueghörig ist. Da die Aufgaben später die Vorgabe haben sollen, dass Wörter einer ganz konkreten Wortklasse verwendet sollen, werden in diesem Schritt alle Wörter entfernt die in weniger als 90% der Fälle in der Hauptklasse auftreten. Der Wert von 90% wurde hierbei durch testen ermittelt und hat die besten Ergebnisse erzielt.

In [4]:
df = df[(df['Percentage_dom_PoS'] > 0.9)]

# 4. Normalisieren des SUBTLCD Werts
Um im späteren Verlauf besser mit dem Wert der Auftrittswahrscheinlichkeit rechnen zu können wird in diesem Schritt der Wert normalisiert (0 bis 1.0).

In [5]:
# normalize SUBTLCD
df['SUBTLCD'] = df['SUBTLCD']/100

# 5. Löschen von ungenutzten Spalten und Umbenennen der relevanten Spalten
Für eine bessere Lesbarkeit und Speicherverwaltung werden alle ungenutzten Spalten entfernt, sowie die relevanten Spalten umbenannt.
Übrig bleiben so die Spalte "Word", welche das konkrete Wort enthält, die Spalte "Frequency" (vorher SUBTLCD) welche die Auftrittswahrscheinlichkeit enthält und die Spalte "Type" welche die zueghörige Wortklasse repräsentiert.

In [6]:

df = df[['Word', 'SUBTLCD', 'Dom_PoS_SUBTLEX']].rename(columns={'SUBTLCD':'Frequency', 'Dom_PoS_SUBTLEX':'Type'})

# 6. Normalisieren der Auftrittswahrscheinlichkeit nach der jeweiligen Wortklasse
Da die Auftrittswahrscheinlichkeit wortklassenübergreifend berechnet wurde, gibt es Klassen welche auf Grund ihrer Eigenschaft nur sehr selten in Sätzen verwendet werden. Hierdurch weißen alle zugehörigen Wörter eine niedrige Auftrittswahrscheinlichkeit auf. Um die Auftrittswahrscheinlichkeit einer globalen Schwierigkeit anzunäheren (z.B. Wörter mit einer Frequenz von > 90% sind leicht, Frequnzen von unter 5% hingegen sehr Schwer) werden in diesem Schritt alle Frequenzen so normalisiert, dass jede Wortklasse Frequenzen von 0 bis 1.0 aufweißt.

Hierzu wird zuerst die minale und maximale Frequenz der jeweiligen Klasse als Spalte hinzugefügt (calc_min_max).
Im zweiten Schritt wird dann in einer neuen Spalte "FrequencyType" die Auftrittswahrscheinlichkeit des Worters innerhalb seiner Klasse zwischen 0 (minimal wert) und 1.0 (maximal wert) eingeordnet.

In [7]:
#1 step
def calc_min_max(s):
    type_seq = df[(df['Type'] == s['Type'])]['Frequency']
    s['Min'] = type_seq.min()
    s['Max'] = type_seq.max()
    return s

word_types_frequency_range = pd.DataFrame({'Type': df['Type'].unique()}).apply(calc_min_max, axis=1)

#2 step
def calculate_typed_frequency(s):
    wtfr = word_types_frequency_range[(word_types_frequency_range['Type'] == s['Type'])]
    return ((s['Frequency'] - wtfr['Min'].iloc[0])/(wtfr['Max'].iloc[0] - wtfr['Min'].iloc[0]))

df['FrequencyType'] = df.apply(lambda x: calculate_typed_frequency(x), axis=1)


# 7. Index zurücksetzen
Als letzten Aufbereitungsschritt wird der Index des DataFrames zurückgesetzt, da durch das löschen einzelner Zeilen entsprechend auch deren Indice entfernt wurde (Pandas spezifisch)

In [8]:

df.reset_index(drop=True, inplace=True)

# FINALE FUNKTION ZUM ABRUFEN VON WÖRTER EINER BESTIMMTEN WORTKLASSE

### get( )
Die Funktion kann nun auf der Basis der zuvor bereitgestellten Wortdaten eine bestimmte Anzahl ('amount') an Wörter einer bestimmten Wortklasse ('word_type') zuückgeben. Mit den Parametern 'min_freq' kann zudem die Frquenz nach unten, mit 'max_freq' entsprechen nach oben beschränkt werden.

Will man beispielsweise nur "leichte" Wörter kann die minimale Frequenz auf 0.9 bsechränkt werden. Es werden also nur noch Wörter mit einer Auftrittswahrschienlich von 90% inherhalb ihrer eignenen Wortklasse zurückgegeben.

Will man "mittelschwere" Wörter kann die maximale Frequenz auf 0.9 und die minimale Frequenz auf 0.5 gesetzt werden. Somit werden sehr häufig auftretende Wörter ausgeschlossen (häufiger als 90%), genau wie sehr schwere Wörter (seltener als 50%).

### types( )
Mit der Funktion werden alle im DataFrame verfügbaren Wortklassen zurückgegeben.

In [9]:
def get(word_type: str, amount: int, min_freq: float = 0.0, max_freq: float = 1.0) -> List[str]:
    available_words = df[(df['Type'] == word_type) & (df['FrequencyType'] >= min_freq) & (df['FrequencyType'] <= max_freq)]['Word']
    return [word for word in available_words.sample(amount if len(available_words) >= amount else len(available_words))]

def types() -> List[str]:
    word_types = [t for t in df['Type'].unique()]
    word_types.sort()
    return word_types

In [10]:
#Beispiel (Zelle erneut ausführen um neue Worte zu erhalten)

# 10 "einfache" Adverben
ex1 = get(word_type='Adverb', amount=10, min_freq=0.9)
print('10 easy adverbs: \t' + ', '.join(ex1), end='\n\n')

# 3 "schwere" Nomen
ex2 = get(word_type='Noun', amount=3, max_freq=0.1)
print('3 hard nouns: \t\t' + ', '.join(ex2), end='\n\n')

# 5 "mittel schwere

10 easy adverbs: 	down, then, maybe, here, please, still, why, never, too, again

3 hard nouns: 		stiffeners, superimpositions, pattee

5 middle verbs: 	came, excuse, asked, telling, sit

available word types: 	Adjective, Adverb, Article, Conjunction, Determiner, Interjection, Name, Noun, Number, Preposition, Pronoun, Verb



In [ ]:
" Verben
ex3 = get(word_type='Verb', amount=5, max_freq=0.9, min_freq=0.5)
print('5 middle verbs: \t' + ', '.join(ex3), end='\n\n')

# Verfügbare Wortklassen
ex4 = types()
print('available word types: \t' + ', '.join(ex4), end='\n\n')